In [2]:
from skimage.io import imread_collection
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Reading in the data

In [3]:
pca_flat = pd.read_csv("pca_selfies.csv") 
pca_flat.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,154,155,156,157,158,159,160,161,position,label
0,0,441.932661,7952.285956,-6597.443229,4630.249901,13140.750464,12722.120890,11494.439900,-8702.121877,-9162.197450,...,-5.406816,-0.271236,-3.931996,0.467108,0.837577,-0.563901,-0.942227,0.168963,0.0,1
1,1,8989.608558,11673.484953,1131.446362,1844.040323,-8753.324088,1742.046292,-5237.215777,3268.983367,-2904.885423,...,2.564581,-1.487472,0.601396,0.160433,-5.907654,-1.736275,-1.412627,-1.312221,0.0,1
2,2,10668.507770,12591.264090,986.757039,5048.943308,-8241.432345,693.392980,-2488.998214,1838.418512,125.335751,...,-0.353201,-2.309761,3.313154,-0.944603,1.756764,-10.075806,-4.374063,2.741708,0.0,1
3,3,11714.634149,12304.238553,1232.151830,6968.885752,-7723.520810,230.108391,-448.051474,390.515751,2478.778945,...,3.839871,-1.356766,-2.746948,-2.019972,-10.868602,1.781758,-0.310839,-0.880320,0.0,1
4,4,12292.692526,11911.251073,987.619597,8391.517959,-6527.802073,-187.887713,663.867692,-347.012928,3296.444577,...,-3.333900,-10.156313,2.697583,6.331006,-2.525387,0.411994,19.349089,-2.866200,0.0,1


In [4]:
pca_flat.shape

(168, 165)

# Random Forest Model

## Test/train split 

In [23]:
from sklearn.model_selection import train_test_split

X = pca_flat.drop(['position', 'label'], axis=1)
y = pca_flat['label']
strat = pca_flat[['position', 'label']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.285, stratify=strat, random_state=101)

In [24]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(168, 163)
(120, 163)
(48, 163)


## Unscaled 

### Simple Model 

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV

In [26]:
rf = RandomForestClassifier(random_state=101)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [27]:
print(accuracy_score(y_test, y_pred_rf))

0.9791666666666666


In [28]:
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           1       0.94      1.00      0.97        16
           2       1.00      0.94      0.97        16
           3       1.00      1.00      1.00        16

   micro avg       0.98      0.98      0.98        48
   macro avg       0.98      0.98      0.98        48
weighted avg       0.98      0.98      0.98        48



In [29]:
print(confusion_matrix(y_test,y_pred_rf))

[[16  0  0]
 [ 1 15  0]
 [ 0  0 16]]


###  Grid Search

In [30]:
param_grid = {'n_estimators': [10, 25, 50, 100], 
              'max_depth': [10, 15, 30, 50],
              'min_samples_split':[2, 10, 50, 100], 
              'min_impurity_decrease': [.0000001, .00001, .001, .1]} 

In [13]:
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3)
grid.fit(X_train, y_train)

C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 256 candidates, totalling 768 fits
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10, score=0.7619047619047619, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10, score=0.9230769230769231, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10, score=0.9487179487179487, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=25 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=25, score=0.9047619047619048, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=25 

[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=100 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=100 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=2, n_estimators=10, score=0.9285714285714286, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=2, n_estimators=10, score=0.8461538461538461, total=   0.0s
[CV] m

[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=100 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=100 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=2, n_estimators=10, score=0.9285714285714286, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=2, n_estimators=10, score=0.8974358974358975, total=   0.0s
[CV] m

[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=100, n_estimators=100 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=100, n_estimators=100 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=10, score=0.8333333333333334, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=10, score=0.7692307692307693, total=   0.0s
[CV] max_depth

...

[CV]  max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=50, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=50, score=0.7692307692307693, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=50, score=0.8717948717948718, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=100, score=0.9523809523809523, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=100, score=0.8717948717948718, total=   0.0s
[CV] max_depth=50, min_

[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25, score=0.6923076923076923, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25, score=0.9487179487179487, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50, score=0.8333333333333334, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50, score=0.8717948717948718, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50, score=1.0, total=   0.0s
[CV] max_depth=50, min_impu

[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=10, n_estimators=50, score=0.8974358974358975, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=10, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=10, n_estimators=100, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=10, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=10, n_estimators=100, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=10, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=10, n_estimators=100, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=50, n_estimators=10 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=50, n_estimators=10, score=0.2619047619047619, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=50, n

[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:   27.1s finished
C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 25, 50, 100], 'max_depth': [10, 15, 30, 50], 'min_samples_split': [2, 10, 50, 100], 'min_impurity_decrease': [1e-07, 1e-05, 0.001, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [14]:
print(grid.best_params_)
print(grid.best_estimator_)

{'max_depth': 10, 'min_impurity_decrease': 1e-07, 'min_samples_split': 2, 'n_estimators': 100}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=1e-07, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [15]:
grid_predictions = grid.predict(X_test)
print(confusion_matrix(y_test, grid_predictions))

[[16  0  0]
 [ 0 16  0]
 [ 0  0 16]]


## Scaled

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [39]:
scaler = StandardScaler()
scaler.fit(pca_flat.drop(['position', 'label'],axis=1))

scaled_features = scaler.transform(pca_flat.drop(['position', 'label'],axis=1))
scaled_features


C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


array([[-1.72177148e+00,  4.38892273e-02,  8.70871736e-01, ...,
        -4.85773756e-03, -8.38895608e-03,  1.59222808e-03],
       [-1.70115147e+00,  8.92776226e-01,  1.27838815e+00, ...,
        -1.49571771e-02, -1.25770761e-02, -1.23657820e-02],
       [-1.68053145e+00,  1.05951111e+00,  1.37889609e+00, ...,
        -8.67982177e-02, -3.89437033e-02,  2.58366278e-02],
       ...,
       [ 1.68053145e+00, -4.53902874e-01, -7.15345135e-01, ...,
         7.64116982e-02, -9.96055169e-03, -7.39056788e-02],
       [ 1.70115147e+00, -1.24361441e+00,  3.92508999e-03, ...,
        -5.65522276e-02,  2.14103628e-02, -1.92167089e-02],
       [ 1.72177148e+00, -4.47539926e-01, -6.51914020e-01, ...,
        -4.22774521e-03, -1.90278913e-01,  1.55638804e-01]])

In [40]:
y = pca_flat['label']
strat = pca_flat[['position', 'label']]

X_train, X_test, y_train, y_test = train_test_split(scaled_features, y, test_size=0.285, stratify=strat, 
                                                    random_state=101)

### Simple Model

In [41]:
rf_scaled = RandomForestClassifier(random_state=101)
rf_scaled.fit(X_train, y_train)

y_pred_scaled = rf_scaled.predict(X_test)

C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [42]:
print(accuracy_score(y_test, y_pred_scaled))

0.9791666666666666


In [43]:
print(classification_report(y_test, y_pred_scaled))

              precision    recall  f1-score   support

           1       0.94      1.00      0.97        16
           2       1.00      0.94      0.97        16
           3       1.00      1.00      1.00        16

   micro avg       0.98      0.98      0.98        48
   macro avg       0.98      0.98      0.98        48
weighted avg       0.98      0.98      0.98        48



In [44]:
print(confusion_matrix(y_test, y_pred_scaled))

[[16  0  0]
 [ 1 15  0]
 [ 0  0 16]]


### Grid Search

In [45]:
param_grid = {'n_estimators': [10, 25, 50, 100], 
              'max_depth': [10, 15, 30, 50],
              'min_samples_split':[2, 10, 50, 100], 
              'min_impurity_decrease': [.0000001, .00001, .001, .1]} 

In [46]:
grid_scaled = GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3)
grid_scaled.fit(X_train, y_train)

C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 256 candidates, totalling 768 fits
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10, score=0.8571428571428571, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10, score=0.9230769230769231, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=10, score=0.8717948717948718, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=25 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=25, score=1.0, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=2, n_estimators=25 
[CV]  max_dept

[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=25, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=25 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=25, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=25 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=25, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=50 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=50, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=50 
[CV]  max_depth=10, min_impurity_decrease=1e-07, min_samples_split=100, n_estimators=50, score=0.3333333333333333, total=   0.0s
[CV

[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=100, score=0.9487179487179487, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=100 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=50, n_estimators=100, score=0.9487179487179487, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=10, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=10, score=0.3333333333333333, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=10 
[CV]  max_depth=10, min_impurity_decrease=1e-05, min_samples_split=100, n_estimators=10, score=0.3333333333333333, total=   0.0s
[CV

[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25, score=0.8095238095238095, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25, score=0.6666666666666666, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=25, score=0.9230769230769231, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50, score=0.9047619047619048, total=   0.0s
[CV] max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50 
[CV]  max_depth=10, min_impurity_decrease=0.001, min_samples_split=50, n_estimators=50, score=0.7435897435897436, total=   0.0s
[CV] max_dep

...

[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=2, n_estimators=100, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=2, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=2, n_estimators=100, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=10, n_estimators=10 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=10, n_estimators=10, score=0.9761904761904762, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=10, n_estimators=10 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=10, n_estimators=10, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_samples_split=10, n_estimators=10 
[CV]  max_depth=50, min_impurity_decrease=0.001, min_samples_split=10, n_estimators=10, score=0.9487179487179487, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.001, min_sampl

[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=25, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=50, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=50, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=50 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=50, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=100, score=1.0, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=2, n_estimators=100 
[CV]  max_depth=50, min_imp

[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s
[CV] max_depth=50, min_impurity_decrease=0.1, min_samples_split=100, n_estimators=100 
[CV]  max_depth=50, min_impurity_decrease=0.1, min_samples_split=100, n_estimators=100, score=0.3333333333333333, total=   0.0s


[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:   26.2s finished
C:\Users\kco6659\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 25, 50, 100], 'max_depth': [10, 15, 30, 50], 'min_samples_split': [2, 10, 50, 100], 'min_impurity_decrease': [1e-07, 1e-05, 0.001, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [47]:
print(grid_scaled.best_params_)
print(grid_scaled.best_estimator_)

{'max_depth': 10, 'min_impurity_decrease': 1e-07, 'min_samples_split': 2, 'n_estimators': 100}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=1e-07, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [48]:
grid_pred_scaled = grid_scaled.predict(X_test)
print(confusion_matrix(y_test, grid_pred_scaled))

[[16  0  0]
 [ 0 16  0]
 [ 0  0 16]]
